# Importamos e instalamos las librerias necesarias

In [1]:
!pip install selenium
import folium
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Definimos nuestra clase optimizadora de Colonia de Hormigas.

In [4]:
# Sacado de: https://github.com/johnberroa?tab=repositories

warnings.filterwarnings("ignore")


class AntColonyOptimizer:
    def __init__(self, ants, evaporation_rate, intensification, alpha=1.0, beta=0.0, beta_evaporation_rate=0,
                 choose_best=.1):
        """
        Ant colony optimizer.  Traverses a graph and finds either the max or min distance between nodes.
        :param ants: number of ants to traverse the graph
        :param evaporation_rate: rate at which pheromone evaporates
        :param intensification: constant added to the best path
        :param alpha: weighting of pheromone
        :param beta: weighting of heuristic (1/distance)
        :param beta_evaporation_rate: rate at which beta decays (optional)
        :param choose_best: probability to choose the best route
        """
        # Parameters
        self.ants = ants
        self.evaporation_rate = evaporation_rate
        self.pheromone_intensification = intensification
        self.heuristic_alpha = alpha
        self.heuristic_beta = beta
        self.beta_evaporation_rate = beta_evaporation_rate
        self.choose_best = choose_best

        # Internal representations
        self.pheromone_matrix = None
        self.heuristic_matrix = None
        self.probability_matrix = None

        self.map = None
        self.set_of_available_nodes = None

        # Internal stats
        self.best_series = []
        self.best = None
        self.fitted = False
        self.best_path = None
        self.fit_time = None

        # Plotting values
        self.stopped_early = False

    def __str__(self):
        string = "Ant Colony Optimizer"
        string += "\n--------------------"
        string += "\nDesigned to optimize either the minimum or maximum distance between nodes in a square matrix that behaves like a distance matrix."
        string += "\n--------------------"
        string += "\nNumber of ants:\t\t\t\t{}".format(self.ants)
        string += "\nEvaporation rate:\t\t\t{}".format(self.evaporation_rate)
        string += "\nIntensification factor:\t\t{}".format(self.pheromone_intensification)
        string += "\nAlpha Heuristic:\t\t\t{}".format(self.heuristic_alpha)
        string += "\nBeta Heuristic:\t\t\t\t{}".format(self.heuristic_beta)
        string += "\nBeta Evaporation Rate:\t\t{}".format(self.beta_evaporation_rate)
        string += "\nChoose Best Percentage:\t\t{}".format(self.choose_best)
        string += "\n--------------------"
        string += "\nUSAGE:"
        string += "\nNumber of ants influences how many paths are explored each iteration."
        string += "\nThe alpha and beta heuristics affect how much influence the pheromones or the distance heuristic weigh an ants' decisions."
        string += "\nBeta evaporation reduces the influence of the heuristic over time."
        string += "\nChoose best is a percentage of how often an ant will choose the best route over probabilistically choosing a route based on pheromones."
        string += "\n--------------------"
        if self.fitted:
            string += "\n\nThis optimizer has been fitted."
        else:
            string += "\n\nThis optimizer has NOT been fitted."
        return string

    def _initialize(self):
        """
        Initializes the model by creating the various matrices and generating the list of available nodes
        """
        assert self.map.shape[0] == self.map.shape[1], "Map is not a distance matrix!"
        num_nodes = self.map.shape[0]
        self.pheromone_matrix = np.ones((num_nodes, num_nodes))
        # Remove the diagonal since there is no pheromone from node i to itself
        self.pheromone_matrix[np.eye(num_nodes) == 1] = 0
        self.heuristic_matrix = 1 / self.map
        self.probability_matrix = (self.pheromone_matrix ** self.heuristic_alpha) * (
                self.heuristic_matrix ** self.heuristic_beta)  # element by element multiplcation
        self.set_of_available_nodes = list(range(num_nodes))

    def _reinstate_nodes(self):
        """
        Resets available nodes to all nodes for the next iteration
        """
        self.set_of_available_nodes = list(range(self.map.shape[0]))

    def _update_probabilities(self):
        """
        After evaporation and intensification, the probability matrix needs to be updated.  This function
        does that.
        """
        self.probability_matrix = (self.pheromone_matrix ** self.heuristic_alpha) * (
                self.heuristic_matrix ** self.heuristic_beta)

    def _choose_next_node(self, from_node):
        """
        Chooses the next node based on probabilities.  If p < p_choose_best, then the best path is chosen, otherwise
        it is selected from a probability distribution weighted by the pheromone.
        :param from_node: the node the ant is coming from
        :return: index of the node the ant is going to
        """
        numerator = self.probability_matrix[from_node, self.set_of_available_nodes]
        if np.random.random() < self.choose_best:
            next_node = np.argmax(numerator)
        else:
            denominator = np.sum(numerator)
            probabilities = numerator / denominator
            next_node = np.random.choice(range(len(probabilities)), p=probabilities)
        return next_node

    def _remove_node(self, node):
        self.set_of_available_nodes.remove(node)

    def _evaluate(self, paths, mode):
        """
        Evaluates the solutions of the ants by adding up the distances between nodes.
        :param paths: solutions from the ants
        :param mode: max or min
        :return: x and y coordinates of the best path as a tuple, the best path, and the best score
        """
        scores = np.zeros(len(paths))
        coordinates_i = []
        coordinates_j = []
        for index, path in enumerate(paths):
            score = 0
            coords_i = []
            coords_j = []
            for i in range(len(path) - 1):
                coords_i.append(path[i])
                coords_j.append(path[i + 1])
                score += self.map[path[i], path[i + 1]]
            scores[index] = score
            coordinates_i.append(coords_i)
            coordinates_j.append(coords_j)
        if mode == 'min':
            best = np.argmin(scores)
        elif mode == 'max':
            best = np.argmax(scores)
        return (coordinates_i[best], coordinates_j[best]), paths[best], scores[best]

    def _evaporation(self):
        """
        Evaporate some pheromone as the inverse of the evaporation rate.  Also evaporates beta if desired.
        """
        self.pheromone_matrix *= (1 - self.evaporation_rate)
        self.heuristic_beta *= (1 - self.beta_evaporation_rate)

    def _intensify(self, best_coords):
        """
        Increases the pheromone by some scalar for the best route.
        :param best_coords: x and y (i and j) coordinates of the best route
        """
        i = best_coords[0]
        j = best_coords[1]
        self.pheromone_matrix[i, j] += self.pheromone_intensification

    def fit(self, map_matrix, iterations=100, mode='min', early_stopping_count=50, verbose=True):
        """
        Fits the ACO to a specific map.  This was designed with the Traveling Salesman problem in mind.
        :param map_matrix: Distance matrix or some other matrix with similar properties
        :param iterations: number of iterations
        :param mode: whether to get the minimum path or maximum path
        :param early_stopping_count: how many iterations of the same score to make the algorithm stop early
        :return: the best score
        """
        if verbose: print("Beginning ACO Optimization with {} iterations...".format(iterations))
        self.map = map_matrix
        start = time.time()
        self._initialize()
        num_equal = 0

        for i in range(iterations):
            start_iter = time.time()
            paths = []
            path = []

            for ant in range(self.ants):
                current_node = self.set_of_available_nodes[0] # Definimos el nodo 0 como el nodo inicial.
                start_node = current_node
                while True:
                    path.append(current_node)
                    self._remove_node(current_node)
                    if len(self.set_of_available_nodes) != 0:
                        current_node_index = self._choose_next_node(current_node)
                        current_node = self.set_of_available_nodes[current_node_index]
                    else:
                        break

                path.append(start_node)  # go back to start
                self._reinstate_nodes()
                paths.append(path)
                path = []

            best_path_coords, best_path, best_score = self._evaluate(paths, mode)

            if i == 0:
                best_score_so_far = best_score
            else:
                if mode == 'min':
                    if best_score < best_score_so_far:
                        best_score_so_far = best_score
                        self.best_path = best_path
                elif mode == 'max':
                    if best_score > best_score_so_far:
                        best_score_so_far = best_score
                        self.best_path = best_path

            if best_score == best_score_so_far:
                num_equal += 1
            else:
                num_equal = 0

            self.best_series.append(best_score)
            self._evaporation()
            self._intensify(best_path_coords)
            self._update_probabilities()

            if verbose: print("Best score at iteration {}: {}; overall: {} ({}s)"
                              "".format(i, round(best_score, 2), round(best_score_so_far, 2),
                                        round(time.time() - start_iter)))

            if best_score == best_score_so_far and num_equal == early_stopping_count:
                self.stopped_early = True
                print("Stopping early due to {} iterations of the same score.".format(early_stopping_count))
                break

        self.fit_time = round(time.time() - start)
        self.fitted = True

        if mode == 'min':
            self.best = self.best_series[np.argmin(self.best_series)]
            if verbose: print(
                "ACO fitted.  Runtime: {} minutes.  Best score: {}".format(self.fit_time // 60, self.best))
            return self.best
        elif mode == 'max':
            self.best = self.best_series[np.argmax(self.best_series)]
            if verbose: print(
                "ACO fitted.  Runtime: {} minutes.  Best score: {}".format(self.fit_time // 60, self.best))
            return self.best
        else:
            raise ValueError("Invalid mode!  Choose 'min' or 'max'.")

    def plot(self):
        """
        Plots the score over time after the model has been fitted.
        :return: None if the model isn't fitted yet
        """
        if not self.fitted:
            print("Ant Colony Optimizer not fitted!  There exists nothing to plot.")
            return None
        else:
            fig, ax = plt.subplots(figsize=(20, 15))
            ax.plot(self.best_series, label="Best Run")
            ax.set_xlabel("Iteration")
            ax.set_ylabel("Performance")
            ax.text(.8, .6,
                    'Ants: {}\nEvap Rate: {}\nIntensify: {}\nAlpha: {}\nBeta: {}\nBeta Evap: {}\nChoose Best: {}\n\nFit Time: {}m{}'.format(
                        self.ants, self.evaporation_rate, self.pheromone_intensification, self.heuristic_alpha,
                        self.heuristic_beta, self.beta_evaporation_rate, self.choose_best, self.fit_time // 60,
                        ["\nStopped Early!" if self.stopped_early else ""][0]),
                    bbox={'facecolor': 'gray', 'alpha': 0.8, 'pad': 10}, transform=ax.transAxes)
            ax.legend()
            plt.title("Ant Colony Optimization Results (best: {})".format(np.round(self.best, 2)))
            plt.show()




# Creamos las matrices necesarias para la resolución del problema de nuestro vendedor.

In [5]:


# Crear una matriz de distancias entre cada ciudad
distancias = np.array([[0, 401, 79.2, 438, 194, 165, 241, 1102, 806, 1234, 1031, 1237, 401, 752, 945],
                      [401 , 0, 467, 845, 582, 559, 424, 1493, 1203, 1623, 1434, 1631, 802, 1140, 1329],
                      [79.2 , 467, 0, 361, 118, 88.7, 170, 1031, 726, 1156, 961, 1165, 329, 674, 865],
                      [438 , 845, 361, 0, 318, 282, 302, 867, 832, 995, 1050, 979, 443, 407, 557],
                      [194 , 582, 118, 318, 0, 48, 54, 913, 621, 1042, 849, 1050, 215, 559, 747],
                      [165 , 559, 88.7, 282, 48, 0, 39, 949, 663, 1082, 903, 968, 268, 594, 804],
                      [241 , 424, 170, 302, 54, 39, 0, 860, 600, 993, 831, 894, 194, 508, 694],
                      [1102, 1493, 1031, 867, 913, 949, 860, 0, 433, 296, 365, 295, 745, 449, 525],
                      [806, 1203, 726, 832, 621, 663, 600, 433, 0, 343, 276, 353, 405, 717, 795],
                      [1234, 1623, 1156, 995, 1042, 1082, 993, 296, 343, 0, 129, 16.2, 696, 580, 670],
                      [1031, 1434, 961, 1050, 849, 903, 831, 365, 276, 129, 0, 120, 640, 650, 731],
                      [1237, 1631, 1165, 979, 1050, 968, 894, 295, 353, 16.2, 120, 0, 702, 585, 657],
                      [401 , 802, 329, 443, 215, 268, 194, 745, 405, 696, 640, 702, 0, 404, 598],
                      [752, 1140, 674, 407, 559, 594, 508, 449, 717, 580, 650, 585, 404, 0, 198],
                      [945, 1329, 865, 557, 747,804, 694, 525, 795, 670, 731, 657, 598, 198, 0]])

# Crear una matriz del valor total de peajes entre cada ciudad
peajes = np.array([[0, 46600, 27500, 99600, 49600, 47900, 75700, 160000, 170300, 213900, 203600, 175900, 93000, 122800, 140100],
                  [46600, 0, 55900, 133900, 80300, 78500, 104700, 183400, 199200, 233500, 229500, 222200, 122500, 156000, 171100],
                  [27500, 55900, 0, 144200, 22100, 40800, 57300, 141600, 142800, 186400, 183400, 267800, 74600, 328900, 129500],
                  [99600, 133900, 144200, 0, 70700, 55400, 45400, 103700, 137900, 139000, 130800, 142500, 74800, 55100, 42600],
                  [49600, 80300, 22100, 70700, 0, 15300, 24400, 103100, 118900, 164300, 149200, 141900, 42200, 75700, 90800],
                  [47900, 78500, 40800, 55400, 15300, 0, 39700, 100000, 134200, 138100, 164500, 138800, 57500, 72600, 87700],
                  [75700, 104700, 57300, 45400, 24400, 39700, 0, 78700, 106100, 118100, 136400, 117500, 29400, 51300, 66400],
                  [160000, 183400, 141600, 103700, 103100, 100000, 78700, 0, 47800, 48700, 37600, 49300, 97500, 49300, 29900],
                  [170300, 199200, 142800, 137900, 118900, 134200, 106100, 47800, 0, 40700, 54500, 41000, 76700, 86600, 67200],
                  [213900, 233500, 186400, 139000, 164300, 138100, 118100, 48700, 40700, 0, 24900, 0, 122100, 85300, 66900],
                  [203600, 229500, 183400, 130800, 149200, 164500, 136400, 37600, 54500, 24900, 0, 33500, 107000, 76400, 57000],
                  [175900, 222200, 267800, 142500, 141900, 138800, 117500, 49300, 41000, 0, 33500, 0, 112400, 88100, 68700],
                  [93000, 122500, 74600, 74800, 42200, 57500, 29400, 97500, 76700, 122100, 107000, 112400, 0, 70100, 85200],
                  [122800, 156000, 328900, 55100, 75700, 72600, 51300, 49300, 86600, 85300, 76400, 88100, 70100, 0, 15100],
                  [140100, 171100, 129500, 42600, 90800, 87700, 66400, 29900, 67200, 66900, 57000, 68700, 85200, 15100, 0]])


# Declaramos todas las variables que usaremos

In [6]:
consumoKwid = 0.0139
galonGasolina = 10766
velPromedio = 75
horaVendedor = 8000

# Realizaremos todas las operaciones de matrices necesarias para unificar todos los costos del viaje que debe realizar el vendedor.

In [ ]:
galones = distancias * consumoKwid # Es la cantidad de galones consumidos por cada una de las distancias teniendo en cuenta el consumo del renault Kwid
costoGasolina = galones * galonGasolina # Es el costo en gasolina del trayecto por cada distancia
costoViaje = costoGasolina + peajes # Es el costo de la gasolina por cada trayecto más el valor total de los peajes entre cada ciudad
sueldoVendedor = (distancias/velPromedio) * horaVendedor # Es el tiempo que se tarda en recorrer cada trayecto a una velocidad promedio de 75 km/hora multiplicado por el valor de la hora del vendedor
costoTotal = costoViaje + sueldoVendedor

Realizamos la optimización con colonia de hormigas, buscamos la mejor ruta y el menor costo.

In [9]:
# Asignación de la matriz con los costos entre cada ciudad a la variable map
map = costoTotal

# Crear una instancia del optimizador de colonia de hormigas
#optimizer = AntColonyOptimizer(ants=100, evaporation_rate=0.5, intensification=2.0, alpha=1.0, beta=1.0, choose_best=0.1)
optimizer = AntColonyOptimizer(ants=100, evaporation_rate=0.1, intensification=2.0, alpha=1.0, beta=1.0, choose_best=0.1)

# Asignar la matriz de distancias al optimizador
optimizer.map = map

# Inicializar el optimizador
optimizer._initialize()

# Ejecutar el algoritmo de optimización
optimizer.fit(map, mode='min') # Buscamos minimizar

# Obtener la mejor ruta y la distancia mínima
best_path = optimizer.best_path
cost = optimizer.best

# Imprimir la mejor ruta y la distancia mínima
print("Mejor Camino: ", best_path)
print("Costo del viaje: ", cost)

Beginning ACO Optimization with 100 iterations...
Best score at iteration 0: 1890614.97; overall: 1890614.97 (0s)
Best score at iteration 1: 1797650.11; overall: 1797650.11 (0s)
Best score at iteration 2: 1797650.11; overall: 1797650.11 (0s)
Best score at iteration 3: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 4: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 5: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 6: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 7: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 8: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 9: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 10: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 11: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 12: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 13: 1657176.72; overall: 1657176.72 (0s)
Best score at iteration 14: 1657176.72; overall: 1

# Graficamos el recorrido realizado por el vendedor

In [10]:
# Convertir la lista con las ciudades recorridas en orden a una lista los números en texto
ciudades = [str(i) for i in best_path]

# Coordenadas de las ciudades
# Enlace de coordenadas: https://www.geodatos.net/coordenadas/colombia/palmira

coordenadas = {
    "0": [3.53944, -76.30361], # Palmira
    "1": [1.21361, -77.28111], # Pasto
    "2": [4.08466, -76.19536], # Tulúa
    "3": [4.60971, -74.08175], # Bogotá
    "4": [4.81333, -75.69611], # Pereira
    "5": [4.53389, -75.68111], # Armenia
    "6": [5.06889, -75.51738], # Manizales
    "7": [10.46314, -73.25322], # Valledupar
    "8": [8.74798, -75.88143], # Monteria
    "9": [10.91843, -74.76459], # Soledad
    "10": [10.39972, -75.51444], # Cartagena
    "11": [10.96854, -74.78132], # Barranquilla
    "12": [6.25184, -75.56359], # Medellin
    "13": [7.12539, -73.1198], # Bucaramanga
    "14": [7.89391, -72.50782] # Cucuta
  }

nombres = {
    "0": 'Palmira',
    "1": 'Pasto',
    "2": 'Tulúa',
    "3": 'Bogotá',
    "4": 'Pereira',
    "5": 'Armenia',
    "6": 'Manizales',
    "7": 'Valledupar',
    "8": 'Monteria',
    "9": 'Soledad',
    "10": 'Cartagena',
    "11": 'Barranquilla',
    "12": 'Medellin',
    "13":'Bucaramanga',
    "14": 'Cucuta'
  }

# Creamos el mapa y lo ubicamos en la ciudad inicial
mapa = folium.Map(location=coordenadas[ciudades[0]], zoom_start=6)

# Agregar un marcador para cada ciudad
for ciudad in ciudades:
  folium.Marker(
      location=coordenadas[ciudad],
      #popup=ciudad,
      tooltip=nombres[ciudad] # Muestra el nombre de cada ciudad al pasar el mouse
  ).add_to(mapa)

# Dibujar la ruta entre las ciudades
ruta = []
for ciudad in ciudades:
    ruta.append(coordenadas[ciudad])
folium.PolyLine(
    locations=ruta,
    color='red'
).add_to(mapa)

# Mostrar el mapa
mapa
